# Employee Turn Over

Problem Statement :

We would like to predict Employee Turn over based on employee application activity.

In [118]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [119]:
PATH_DATA='../../data/raw/'
PATH_DATA_INTERIM='../../data/interim/'

In [120]:

def q50(x):
    return x.quantile(0.5)

def q75(x):
    return x.quantile(0.75)

def get_unique_id(df) :
    return  df['employee'].astype(str)+'_'+df['companyAlias']

## Get The data

preview all table

In [121]:
data_churn=pd.read_csv(PATH_DATA+'churn.csv')
data_comment=pd.read_csv(PATH_DATA+'commentInteractions.csv')
data_votes=pd.read_csv(PATH_DATA+'votes.csv')
data_comment_cln0=pd.read_csv(PATH_DATA+'comments_clean_anonimized.csv')

In [122]:
## Happy today?
data_votes.head()

employee              companyAlias                      voteDate  vote
0        31  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     4
1        33  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     4
2        79  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     4
3        94  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     4
4        16  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     2

In [123]:
data_votes.shape

(221232, 4)

In [124]:
data_comment.head()

employee              companyAlias liked disliked                 commentId
0       307  56aec740f1ef260003e307d6  True    False  58d018d7e010990004e38070
1        36  56aec740f1ef260003e307d6  True    False  58d018d7e010990004e38070
2       276  56aec740f1ef260003e307d6  True    False  58d018d7e010990004e38070
3        24  56aec740f1ef260003e307d6  True    False  58d018d7e010990004e38070
4       382  56aec740f1ef260003e307d6  True    False  58d0179ae010990004e3806d

In [125]:
data_comment.shape

(336960, 5)

In [126]:
data_comment_cln0.head()

employee              companyAlias                 commentId  \
0       307  56aec740f1ef260003e307d6  58d018d7e010990004e38070   
1       382  56aec740f1ef260003e307d6  58d0179ae010990004e3806d   
2       172  56aec740f1ef260003e307d6  58cff8cde010990004e37f6a   
3       135  56aec740f1ef260003e307d6  58cfefeee010990004e37f60   
4       225  56aec740f1ef260003e307d6  58cfd9b4e010990004e37f52   

                                                 txt  likes  dislikes  \
0  **********************************************...    4.0       0.0   
1                      *****************************    1.0       2.0   
2                        ***************************    3.0       0.0   
3                        ***************************    1.0       1.0   
4                  *********************************    3.0       2.0   

                    commentDate  
0  Mon Mar 20 19:00:17 CET 2017  
1  Mon Mar 20 18:55:16 CET 2017  
2  Mon Mar 20 16:44:02 CET 2017  
3  Mon Mar 20 16:06:08 CET 2017  
4  Mon Mar 20 14:30:50 CET 2017

In [127]:
data_comment_cln0.shape

(82756, 7)

In [128]:
data_churn.head()

employee              companyAlias  numVotes         lastParticipationDate  \
0       512  56aec740f1ef260003e307d6         4  Thu Feb 23 12:48:04 CET 2017   
1        -2  56aec740f1ef260003e307d6         0  Wed Jan 18 14:00:55 CET 2017   
2         2  56aec740f1ef260003e307d6        72  Fri Mar 17 01:00:00 CET 2017   
3       487  56aec740f1ef260003e307d6        14  Sat Nov 19 15:02:14 CET 2016   
4         3  56aec740f1ef260003e307d6        22  Thu Feb 16 01:00:00 CET 2017   

   stillExists  
0         True  
1        False  
2         True  
3        False  
4         True

In [129]:
data_churn.shape

(4847, 5)

## Formulate the solutions

    Each table has different level. Company level, Employee level, Comment level, or Votes level. 
    Since our target to create model that predict Employee turn over, then we must create features/predictor on Employee level.
    Also, we must have employee that used for training and employee for testing.
    
    Lets generate employee_target.
    

In [130]:
## One of data with Employee level information is data_churn which also contains the target of our model.
data_churn.head()

employee              companyAlias  numVotes         lastParticipationDate  \
0       512  56aec740f1ef260003e307d6         4  Thu Feb 23 12:48:04 CET 2017   
1        -2  56aec740f1ef260003e307d6         0  Wed Jan 18 14:00:55 CET 2017   
2         2  56aec740f1ef260003e307d6        72  Fri Mar 17 01:00:00 CET 2017   
3       487  56aec740f1ef260003e307d6        14  Sat Nov 19 15:02:14 CET 2016   
4         3  56aec740f1ef260003e307d6        22  Thu Feb 16 01:00:00 CET 2017   

   stillExists  
0         True  
1        False  
2         True  
3        False  
4         True

In [131]:
## Employee Id is not unique since different company may have same employee id
len(data_churn.employee.unique())==len(data_churn)

False

In [132]:
data_churn['unique_employee_ids'] = get_unique_id(data_churn)

In [133]:
len(data_churn.unique_employee_ids.unique())==len(data_churn) ## --> still not unique

False

In [134]:
len(data_churn.unique_employee_ids.unique()),len(data_churn)

(4806, 4847)

In [135]:
## for simplicity --> remove the double
employee_data_cnt = data_churn.groupby('unique_employee_ids').size().sort_values()
employee_not_unique = employee_data_cnt[employee_data_cnt>1].index.tolist()
print(len(employee_not_unique))

data_churn2 = data_churn[~(data_churn.unique_employee_ids.isin(employee_not_unique))]

37


In [136]:
len(data_churn2.unique_employee_ids.unique())==len(data_churn2) ## --> unique already.

True

In [137]:
df_employee_target = data_churn2[['unique_employee_ids','stillExists','companyAlias']]
df_employee_target.head()

unique_employee_ids  stillExists              companyAlias
0  512_56aec740f1ef260003e307d6         True  56aec740f1ef260003e307d6
1   -2_56aec740f1ef260003e307d6        False  56aec740f1ef260003e307d6
2    2_56aec740f1ef260003e307d6         True  56aec740f1ef260003e307d6
4    3_56aec740f1ef260003e307d6         True  56aec740f1ef260003e307d6
5   -4_56aec740f1ef260003e307d6        False  56aec740f1ef260003e307d6

In [138]:
train_size = int(len(df_employee_target)*0.7) ##--> train size
print(train_size)

3338


In [139]:
## Get test and generate flag
employee_train = df_employee_target.sample(train_size,random_state = 103)['unique_employee_ids']
df_employee_target['is_train'] = df_employee_target.unique_employee_ids.isin(employee_train).astype(int)

In [140]:
df_employee_target.head()

unique_employee_ids  stillExists              companyAlias  \
0  512_56aec740f1ef260003e307d6         True  56aec740f1ef260003e307d6   
1   -2_56aec740f1ef260003e307d6        False  56aec740f1ef260003e307d6   
2    2_56aec740f1ef260003e307d6         True  56aec740f1ef260003e307d6   
4    3_56aec740f1ef260003e307d6         True  56aec740f1ef260003e307d6   
5   -4_56aec740f1ef260003e307d6        False  56aec740f1ef260003e307d6   

   is_train  
0         0  
1         1  
2         0  
4         0  
5         1

## Employee Features 1

    use data votes.
    remember --> Central European Time – CET Time Zone / European Central Time (Standard Time)
    remember --> hours only 1 and 2 somehow
    possible features in employee level :

    1. Avg Votes
    2. p50 Votes
    3. p75 Votes
    4. Cnt Votes
    5. Std Votes
    6. Avg Votes Monday
    7. p50 Votes Monday
    8. p75 Votes Monday
    9. Cnt Votes Monday
    10. Std Votes Monday
    11. Avg Votes Friday
    12. p50 Votes Friday
    13. p75 Votes Friday
    14. Cnt Votes Friday
    15. Std Votes Friday



In [141]:
data_votes.head()

employee              companyAlias                      voteDate  vote
0        31  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     4
1        33  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     4
2        79  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     4
3        94  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     4
4        16  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     2

In [142]:
data_votes['unique_employee_ids'] = get_unique_id(data_votes)
data_votes['voteDate2'] = pd.to_datetime(data_votes['voteDate'])

In [143]:
data_votes['day_name'] = data_votes['voteDate2'].dt.strftime('%a')
data_votes['votes_monday'] = np.where(data_votes['day_name']=='Mon',data_votes['vote'],np.NaN)
data_votes['votes_friday'] = np.where(data_votes['day_name']=='Fri',data_votes['vote'],np.NaN)

In [144]:
data_votes.head()

employee              companyAlias                      voteDate  vote  \
0        31  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     4   
1        33  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     4   
2        79  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     4   
3        94  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     4   
4        16  56aec740f1ef260003e307d6  Mon Feb 01 01:00:00 CET 2016     2   

           unique_employee_ids           voteDate2 day_name  votes_monday  \
0  31_56aec740f1ef260003e307d6 2016-02-01 01:00:00      Mon           4.0   
1  33_56aec740f1ef260003e307d6 2016-02-01 01:00:00      Mon           4.0   
2  79_56aec740f1ef260003e307d6 2016-02-01 01:00:00      Mon           4.0   
3  94_56aec740f1ef260003e307d6 2016-02-01 01:00:00      Mon           4.0   
4  16_56aec740f1ef260003e307d6 2016-02-01 01:00:00      Mon           2.0   

   votes_friday  
0           NaN  
1           NaN  
2           NaN  
3           NaN  
4           NaN

In [145]:
dg1 = data_votes.groupby('unique_employee_ids')

In [146]:
## group metrics
fun1 = ['count', np.mean, np.std, q50, q75]
fun1_dict = {}
col_f1 = ['unique_employee_ids']
for cols in ['vote', 'votes_monday', 'votes_friday']:
    fun1_dict[cols] = fun1
    for met in ['cnt','avg','std','p50','p75']:
        col_f1.append(cols+'_'+met)
        
emp_f1 = dg1.agg(fun1_dict).reset_index()
emp_f1.columns = col_f1

In [147]:
emp_f1.sample(8)

unique_employee_ids  vote_cnt  vote_avg  vote_std  vote_p50  \
2314  306_57ac8b23be7fe30003e656d0         1  3.000000       NaN       3.0   
2899   42_581b08041a0ef8000308aef6         8  2.250000  0.886405       2.5   
3982   82_574c5ade56b6300003009965         4  2.500000  1.290994       2.5   
2209  291_57d956302a040a00036a8905        76  4.000000  0.000000       4.0   
3203  511_56aec740f1ef260003e307d6        20  3.400000  0.994723       4.0   
1353   19_54d43612e4b0f6a40755d93e        29  3.310345  1.168132       4.0   
62    104_57ac8b23be7fe30003e656d0        16  3.562500  0.629153       4.0   
3073   47_57d1eb86a22c9d0003dd1f05       119  1.521008  0.635780       1.0   

      vote_p75  votes_monday_cnt  votes_monday_avg  votes_monday_std  \
2314      3.00                 0               NaN               NaN   
2899      3.00                 2          3.000000          0.000000   
3982      3.25                 0               NaN               NaN   
2209      4.00                14          4.000000          0.000000   
3203      4.00                 3          3.333333          1.154701   
1353      4.00                 4          3.750000          0.500000   
62        4.00                 3          4.000000          0.000000   
3073      2.00                22          1.272727          0.455842   

      votes_monday_p50  votes_monday_p75  votes_friday_cnt  votes_friday_avg  \
2314               NaN               NaN                 0               NaN   
2899               3.0              3.00                 1          2.000000   
3982               NaN               NaN                 1          1.000000   
2209               4.0              4.00                12          4.000000   
3203               4.0              4.00                 4          3.000000   
1353               4.0              4.00                 2          3.000000   
62                 4.0              4.00                 0               NaN   
3073               1.0              1.75                23          1.695652   

      votes_friday_std  votes_friday_p50  votes_friday_p75  
2314               NaN               NaN               NaN  
2899               NaN               2.0               2.0  
3982               NaN               1.0               1.0  
2209          0.000000               4.0               4.0  
3203          1.414214               3.5               4.0  
1353          1.414214               3.0               3.5  
62                 NaN               NaN               NaN  
3073          0.764840               2.0               2.0

## Employee Features 2

    use data comment cln.
    Acpt likes --> likes > 0 
    possible features in employee level :

    1. Avg long text
    2. p50 long text
    3. p75 long text
    4. std long text
    5. cnt text
    6. Avg Acpt likes
    7. p50 Acpt likes
    8. p75 Acpt likes
    9. std Acpt likes
    10. cnt Acpt dislikes
    11. Avg Acpt dislikes
    12. p50 Acpt dislikes
    13. p75 Acpt dislikes
    14. std Acpt dislikes
    15. cnt Acpt dislikes
    16. cnt morning comment
    17. cnt night comment
    18. cnt lunch comment
    19. cnt weekend comment
    20. cnt weekday comment


In [148]:
data_comment_cln0[data_comment_cln0.commentDate.isna()]

employee              companyAlias                 commentId  txt  \
82755       361  58a728a0e75bda00042a3468  58adb214b8242400048bf8a6  NaN   

       likes  dislikes commentDate  
82755    NaN       NaN         NaN

In [149]:
data_comment_cln1 = data_comment_cln0[~(data_comment_cln0.commentDate.isna())]

In [150]:
data_comment_cln1.head()

employee              companyAlias                 commentId  \
0       307  56aec740f1ef260003e307d6  58d018d7e010990004e38070   
1       382  56aec740f1ef260003e307d6  58d0179ae010990004e3806d   
2       172  56aec740f1ef260003e307d6  58cff8cde010990004e37f6a   
3       135  56aec740f1ef260003e307d6  58cfefeee010990004e37f60   
4       225  56aec740f1ef260003e307d6  58cfd9b4e010990004e37f52   

                                                 txt  likes  dislikes  \
0  **********************************************...    4.0       0.0   
1                      *****************************    1.0       2.0   
2                        ***************************    3.0       0.0   
3                        ***************************    1.0       1.0   
4                  *********************************    3.0       2.0   

                    commentDate  
0  Mon Mar 20 19:00:17 CET 2017  
1  Mon Mar 20 18:55:16 CET 2017  
2  Mon Mar 20 16:44:02 CET 2017  
3  Mon Mar 20 16:06:08 CET 2017  
4  Mon Mar 20 14:30:50 CET 2017

In [151]:
data_comment_cln1['unique_employee_ids'] = get_unique_id(data_comment_cln1)
data_comment_cln1['commentDate2'] = pd.to_datetime(data_comment_cln1['commentDate'])
data_comment_cln1['len_txt'] = data_comment_cln1['txt'].str.len()

In [152]:
# data_comment_cln1[data_comment_cln1.len_txt==0]

In [153]:
data_comment_cln1['is_weekend'] = data_comment_cln1['commentDate2'].dt.strftime('%w').isin(['0','6']).astype(int)
data_comment_cln1['hours'] = data_comment_cln1['commentDate2'].dt.strftime('%H').astype(int)

data_comment_cln1['comment_weekend'] = np.where(data_comment_cln1['is_weekend']==1,data_comment_cln1['txt'],np.NaN)
data_comment_cln1['comment_weekday'] = np.where(data_comment_cln1['is_weekend']==0,data_comment_cln1['txt'],np.NaN)
data_comment_cln1['comment_morning'] = np.where((data_comment_cln1['hours']>=5)&
                                                (data_comment_cln1['hours']<10),data_comment_cln1['txt'],np.NaN)
data_comment_cln1['comment_night'] = np.where((data_comment_cln1['hours']>=19)&
                                                (data_comment_cln1['hours']<=23),data_comment_cln1['txt'],np.NaN)
data_comment_cln1['comment_lunch'] = np.where((data_comment_cln1['hours']>=11)&
                                                (data_comment_cln1['hours']<14),data_comment_cln1['txt'],np.NaN)

data_comment_cln1['acpt_likes'] = np.where(data_comment_cln1.likes>0,data_comment_cln1.likes,np.NaN)
data_comment_cln1['acpt_dislikes'] = np.where(data_comment_cln1.dislikes>0,data_comment_cln1.likes,np.NaN)

In [154]:
data_comment_cln1.sample(8)

employee              companyAlias                 commentId  \
61899        22  574c5ade56b6300003009965  57c03dcb8e0e090003ab596a   
68296       227  57ac8b23be7fe30003e656d0  589de07503f2e60004fff12f   
1596        294  56aec740f1ef260003e307d6  583867cf8e84e80004db64c6   
8012         21  56aec740f1ef260003e307d6  57d663d27793290003d3df2b   
76781       144  57d956302a040a00036a8905  583060fbc79a6b000474fe50   
72939        40  5742d699f839a10003a407d2  5789f0d837c59500038cc6d1   
24183       166  574c423856b6300003009953  58adbe23b8242400048bf8c0   
51960        50  56fd2b64f41c670003f643c8  571e475462947b0003763609   

                                                     txt  likes  dislikes  \
61899  **********************************************...    2.0       1.0   
68296  **********************************************...    0.0       0.0   
1596                        ****************************    8.0       0.0   
8012                                              ******    1.0       0.0   
76781  **********************************************...   18.0       0.0   
72939  **********************************************...    2.0       4.0   
24183  **********************************************...    8.0       5.0   
51960                     ******************************    1.0       0.0   

                         commentDate           unique_employee_ids  \
61899  Fri Aug 26 15:01:01 CEST 2016   22_574c5ade56b6300003009965   
68296   Fri Feb 10 16:40:44 CET 2017  227_57ac8b23be7fe30003e656d0   
1596    Fri Nov 25 17:33:01 CET 2016  294_56aec740f1ef260003e307d6   
8012   Mon Sep 12 10:14:01 CEST 2016   21_56aec740f1ef260003e307d6   
76781   Sat Nov 19 09:57:47 CET 2016  144_57d956302a040a00036a8905   
72939  Sat Jul 16 10:30:44 CEST 2016   40_5742d699f839a10003a407d2   
24183   Wed Feb 22 17:36:01 CET 2017  166_574c423856b6300003009953   
51960  Mon Apr 25 18:33:57 CEST 2016   50_56fd2b64f41c670003f643c8   

             commentDate2  len_txt  is_weekend  hours  \
61899 2016-08-26 15:01:01     62.0           0     15   
68296 2017-02-10 16:40:44     99.0           0     16   
1596  2016-11-25 17:33:01     28.0           0     17   
8012  2016-09-12 10:14:01      6.0           0     10   
76781 2016-11-19 09:57:47    451.0           1      9   
72939 2016-07-16 10:30:44     51.0           1     10   
24183 2017-02-22 17:36:01     51.0           0     17   
51960 2016-04-25 18:33:57     30.0           0     18   

                                         comment_weekend  \
61899                                                NaN   
68296                                                NaN   
1596                                                 NaN   
8012                                                 NaN   
76781  **********************************************...   
72939  **********************************************...   
24183                                                NaN   
51960                                                NaN   

                                         comment_weekday  \
61899  **********************************************...   
68296  **********************************************...   
1596                        ****************************   
8012                                              ******   
76781                                                NaN   
72939                                                NaN   
24183  **********************************************...   
51960                     ******************************   

                                         comment_morning comment_night  \
61899                                                NaN           NaN   
68296                                                NaN           NaN   
1596                                                 NaN           NaN   
8012                                                 NaN           NaN   
76781  **********************************************...           NaN   
72

In [155]:
dg2 = data_comment_cln1.groupby('unique_employee_ids')

In [156]:
## group metrics 1
fun2_1 = ['count', np.mean, np.std, q50, q75]
fun2_1_dict = {}
col_f2_1 = ['unique_employee_ids']
for cols in  ['acpt_likes','acpt_dislikes', 'len_txt'] :
    fun2_1_dict[cols] = fun2_1
    for met in ['cnt','avg','std','p50','p75']:
        col_f2_1.append(cols+'_'+met)
        
emp_f2_1 = dg2.agg(fun2_1_dict).reset_index()
emp_f2_1.columns = col_f2_1

In [157]:
## group metrics 1
fun2_2 = ['count']
fun2_2_dict = {}
col_f2_2 = ['unique_employee_ids']
for cols in ['comment_weekend','comment_weekday','comment_morning','comment_night','comment_lunch']:
    fun2_2_dict[cols] = fun2_2
    for met in ['cnt']:
        col_f2_2.append(cols+'_'+met)
        
emp_f2_2 = dg2.agg(fun2_2_dict).reset_index()
emp_f2_2.columns = col_f2_2

In [158]:
emp_f2_1.sample(3)

unique_employee_ids  acpt_likes_cnt  acpt_likes_avg  \
3170    96_574c5ade56b6300003009965               6        3.333333   
2418    47_57d956302a040a00036a8905               6       14.333333   
0     -100_5370af43e4b0cff95558c12a               8        1.500000   

      acpt_likes_std  acpt_likes_p50  acpt_likes_p75  acpt_dislikes_cnt  \
3170        2.250926             3.0            5.25                  2   
2418       11.039324            11.0           23.75                  4   
0           0.534522             1.5            2.00                  4   

      acpt_dislikes_avg  acpt_dislikes_std  acpt_dislikes_p50  \
3170                0.0           0.000000                0.0   
2418               16.0          13.856406               16.0   
0                   1.0           1.154701                1.0   

      acpt_dislikes_p75  len_txt_cnt  len_txt_avg  len_txt_std  len_txt_p50  \
3170                0.0            8    27.750000    22.082637         20.5   
2418               28.0            6   113.666667    82.415209         74.0   
0                   2.0           14    40.285714    27.025426         34.0   

      len_txt_p75  
3170        37.00  
2418       182.75  
0           44.25

In [159]:
emp_f2_2.sample(3)

unique_employee_ids  comment_weekend_cnt  comment_weekday_cnt  \
439   110_5742d699f839a10003a407d2                    2                   10   
940   162_5742d699f839a10003a407d2                    0                    4   
1427  223_5641f96713664c000332c8cd                   14                   24   

      comment_morning_cnt  comment_night_cnt  comment_lunch_cnt  
439                     2                  0                  4  
940                     2                  0                  0  
1427                    4                  8                 12

## Employee Features 3

    use data comment
    possible features in employee level :

    1. cnt gv liked
    2. cnt gv disliked
    3. total reaction


In [160]:
data_comment.head()

employee              companyAlias liked disliked                 commentId
0       307  56aec740f1ef260003e307d6  True    False  58d018d7e010990004e38070
1        36  56aec740f1ef260003e307d6  True    False  58d018d7e010990004e38070
2       276  56aec740f1ef260003e307d6  True    False  58d018d7e010990004e38070
3        24  56aec740f1ef260003e307d6  True    False  58d018d7e010990004e38070
4       382  56aec740f1ef260003e307d6  True    False  58d0179ae010990004e3806d

In [161]:
data_comment['unique_employee_ids'] = get_unique_id(data_comment)

In [162]:
data_comment['gv_liked'] = np.where(data_comment.liked==True,1,0)
data_comment['gv_disliked'] = np.where(data_comment.disliked==True,1,0)

In [163]:
dg3 = data_comment.groupby('unique_employee_ids')

In [164]:
emp_f3 = dg3.agg({'gv_liked':np.sum, 'gv_disliked':np.sum}).reset_index()
emp_f3.columns = ['unique_employee_ids','gv_liked_cnt','gv_disliked_cnt']
emp_f3['reaction_cnt'] = emp_f3['gv_liked_cnt'] + emp_f3['gv_disliked_cnt']

In [165]:
emp_f3.sample(3)

unique_employee_ids  gv_liked_cnt  gv_disliked_cnt  \
1260    18_567011c035dce00003a07fa4             4                0   
71    -150_54e52607e4b01191dc064966            15                5   
3423    93_5742d699f839a10003a407d2            33                2   

      reaction_cnt  
1260             4  
71              20  
3423            35

## Company level features

    collective engagement of all employee on a company may impact employee turn over. Such as :
    
    1. Company Turn over rate
    2. Company Avg Votes
    3. Company p50 Votes
    4. Company p75 Votes
    5. Company Cnt Votes
    6. Company Std Votes
    7. comment per employee (mean)
    8. txt len per employee (mean)
    9. comment per employee (p50)
    10. txt len per employee (p50)
    11. liked per employee (mean)
    12. disliked per employee (mean)
    12. liked per employee (p50)
    13. disliked per employee (p50)
    
    this information is obtained from trained data

In [166]:
## Turn over rate
df_employee_target_tr = df_employee_target[df_employee_target.is_train==1]
comp_f1 = (1-df_employee_target_tr.groupby('companyAlias').stillExists.mean()).reset_index()
comp_f1.columns = ['companyAlias','turn_over_rate']

comp_f1.sample(3)

companyAlias  turn_over_rate
15  573f2c4a3517490003ef7710             0.0
12  56e2a905e3b6fe0003e32855             0.0
27  57e518026d641600035db88a             0.0

In [167]:
data_votes_tr = data_votes[data_votes.unique_employee_ids.isin(employee_train)]

In [168]:
len(data_votes_tr),len(data_votes)

(156281, 221232)

In [169]:
## Votes
dg3 = data_votes_tr.groupby('companyAlias')

fun4 = ['count', np.mean, np.std, q50, q75]
fun4_dict = {}
col_f4 = ['companyAlias']
for cols in ['vote']:
    fun4_dict[cols] = fun4
    for met in ['cnt','avg','std','p50','p75']:
        col_f4.append(cols+'_company_'+met)
        
comp_f2 = dg3.agg(fun4_dict).reset_index()
comp_f2.columns = col_f4

In [170]:
comp_f2.sample(3)

companyAlias  vote_company_cnt  vote_company_avg  \
16  5742d699f839a10003a407d2             17312          2.863390   
11  56aec740f1ef260003e307d6             17973          3.316141   
1   53a2dd43e4b01cc02f1e9011              3038          3.077354   

    vote_company_std  vote_company_p50  vote_company_p75  
16          0.689755                 3                 3  
11          0.948827                 4                 4  
1           1.026296                 3                 4

In [171]:
## comment cln1
data_comment_cln1_tr = data_comment_cln1[data_comment_cln1.unique_employee_ids.isin(employee_train)]

dg4_metric = data_comment_cln1_tr.groupby(['companyAlias','unique_employee_ids']).agg({'len_txt':['count',np.sum]}).reset_index()
dg4_metric.columns = ['companyAlias','unique_employee_ids','total_comment','total_comment_len']
comp_f3 = dg4_metric.groupby('companyAlias').mean().reset_index()
comp_f3.columns = ['companyAlias','comment_per_emp_avg','comment_len_per_emp_avg']
comp_f4 = dg4_metric.groupby('companyAlias').quantile(0.5).reset_index()
comp_f4.columns = ['companyAlias','comment_per_emp_p50','comment_len_per_emp_p50']

print(comp_f3.sample(3))
print(comp_f4.sample(3))

                companyAlias  comment_per_emp_avg  comment_len_per_emp_avg
3   54d43612e4b0f6a40755d93e            26.750000               647.000000
35  58c6e15f32f72a00046f556c             2.882353               234.470588
5   552e2d00e4b066b42fd122ed            13.714286               474.857143
                companyAlias  comment_per_emp_p50  comment_len_per_emp_p50
34  58bf03e5cff4fa0004dd44ef                  2.0                     82.0
7   56558cfd07a5de00030908fb                  2.0                     92.0
4   54e52607e4b01191dc064966                 10.0                    928.0


In [172]:
## comment reaction
data_comment_tr = data_comment[data_comment.unique_employee_ids.isin(employee_train)]
dg5_metric = data_comment_tr.groupby(['companyAlias','unique_employee_ids']).agg({'gv_liked':[np.sum],'gv_disliked':[np.sum]}).reset_index()
dg5_metric.columns = ['companyAlias','unique_employee_ids','gv_liked_exc_cnt','gv_disliked_exc_cnt']

comp_f5 = dg5_metric.groupby('companyAlias').mean().reset_index()
comp_f5.columns = ['companyAlias','gv_liked_per_emp_avg','gv_disliked_per_emp_avg']
comp_f6 = dg5_metric.groupby('companyAlias').quantile(0.5).reset_index()
comp_f6.columns = ['companyAlias','gv_liked_per_emp_p50','gv_disliked_per_emp_p50']

## Merge Features

In [173]:
empl_features = [emp_f1, emp_f2_1, emp_f2_2, emp_f3]
comp_features = [comp_f1, comp_f2, comp_f3, comp_f4, comp_f5, comp_f6]

# all_df_features = empl_features + comp_features

In [174]:
df_interim = df_employee_target.copy()
for d in empl_features:
    df_interim = pd.merge(df_interim, d, on = 'unique_employee_ids', how = 'left')
                          
for d in comp_features:
    df_interim = pd.merge(df_interim, d, on = 'companyAlias', how = 'left')

In [175]:
df_interim

unique_employee_ids  stillExists              companyAlias  \
0     512_56aec740f1ef260003e307d6         True  56aec740f1ef260003e307d6   
1      -2_56aec740f1ef260003e307d6        False  56aec740f1ef260003e307d6   
2       2_56aec740f1ef260003e307d6         True  56aec740f1ef260003e307d6   
3       3_56aec740f1ef260003e307d6         True  56aec740f1ef260003e307d6   
4      -4_56aec740f1ef260003e307d6        False  56aec740f1ef260003e307d6   
...                            ...          ...                       ...   
4764  857_58a728a0e75bda00042a3468         True  58a728a0e75bda00042a3468   
4765  858_58a728a0e75bda00042a3468         True  58a728a0e75bda00042a3468   
4766  859_58a728a0e75bda00042a3468         True  58a728a0e75bda00042a3468   
4767   17_573a0671b5ec330003add34a         True  573a0671b5ec330003add34a   
4768   19_573a0671b5ec330003add34a         True  573a0671b5ec330003add34a   

      is_train  vote_cnt  vote_avg  vote_std  vote_p50  vote_p75  \
0            0       4.0  2.500000  1.290994       2.5      3.25   
1            1       NaN       NaN       NaN       NaN       NaN   
2            0      72.0  2.250000  1.031203       2.0      3.00   
3            0      22.0  3.454545  0.738549       4.0      4.00   
4            1       NaN       NaN       NaN       NaN       NaN   
...        ...       ...       ...       ...       ...       ...   
4764         1       1.0  3.000000       NaN       3.0      3.00   
4765         0       1.0  3.000000       NaN       3.0      3.00   
4766         1       1.0  4.000000       NaN       4.0      4.00   
4767         1       7.0  3.571429  0.534522       4.0      4.00   
4768         1       1.0  4.000000       NaN       4.0      4.00   

      votes_monday_cnt  ...  vote_company_p50  vote_company_p75  \
0                  1.0  ...                 4                 4   
1                  NaN  ...                 4                 4   
2                 13.0  ...                 4                 4   
3                  2.0  ...                 4                 4   
4                  NaN  ...                 4                 4   
...                ...  ...               ...               ...   
4764               0.0  ...                 3                 3   
4765               0.0  ...                 3                 3   
4766               0.0  ...                 3                 3   
4767               2.0  ...                 4                 4   
4768               1.0  ...                 4                 4   

      comment_per_emp_avg  comment_len_per_emp_avg  comment_per_emp_p50  \
0               40.497238              3133.790055                 24.0   
1               40.497238              3133.790055                 24.0   
2               40.497238              3133.790055                 24.0   
3               40.497238              3133.790055                 24.0   
4               40.497238              3133.790055                 24.0   
...                   ...                      ...                  ...   
4764             9.088889               931.844444                  6.5   
4765             9.088889               931.844444                  6.5   
4766             9.088889               931.844444                  6.5   
4767                  NaN                      NaN                  NaN   
4768                  NaN                      NaN                  NaN   

      comment_len_per_emp_p50  gv_liked_per_emp_avg  gv_disliked_per_emp_avg  \
0                      1522.0            105.754902                30.171569   
1                      1522.0            105.754902                30.171569   
2                      1522.0            105.754902                30.171569   
3                      1522.0            105.754902                30.171569   
4                      1522.0            105.754902                30.171569   
...                       ...                   ...                      ...   
4764   

In [177]:
df_interim.to_csv(PATH_DATA_INTERIM+'iter1_interim.csv',index = False)